In [126]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [127]:
!pip install 'kaggle-environments==0.1.6'

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 347.8 kB/s eta 0:00:00a 0:00:01
  Attempting uninstall: kaggle-environments
    Found existing installation: kaggle-environments 1.12.0
    Uninstalling kaggle-environments-1.12.0:
      Successfully uninstalled kaggle-environments-1.12.0


In [128]:
from collections import defaultdict
import numpy as np

In [142]:
from kaggle_environments import evaluate, make, utils

env = make("connectx", debug=True)
env.render()


NameError: name 'kaggle_environments' is not defined

In [130]:
state_key = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]).astype(str) 
print(int(''.join(state_key)) )#空いてる所消す
print(int(''.join(state_key), 3) ) #3進数として解釈して10進に直している
print(hex(int(''.join(state_key), 3)))
s = hex(int(''.join(state_key), 3))[2:]  #0xは切り離す
print(state_key, s)

20000001000000
3189375
0x30aa7f
['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0'
 '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '2' '0' '0' '0' '0' '0' '0' '1'
 '0' '0' '0' '0' '0' '0'] 30aa7f


In [131]:
def my_agent(observation, configuration):
    from random import choice
    return choice([c for c in range(configuration.columns) if observation.board[c] == 0])

In [132]:
print(type(my_agent))

<class 'function'>


In [160]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])

class RandomAgent:
    def __init__(self):
        #self.env = env envはいったん外付けで
        self.gamma = 1
        self.action_size = 7

        random_actions = {0:1/7, 1:1/7, 2:1/7, 3:1/7, 4:1/7, 5:1/7, 6:1/7}

        self.pi = defaultdict(lambda: random_actions)
        self.v = defaultdict(lambda: 0)
        self.cnts = defaultdict(lambda: 0)
        self.memory = [] # state_key, action, reward
        
    def generate_state_key(self, observation):
        state = observation.board
        state.append(observation.mark)
        state_key = np.array(state).astype(str) 

        return hex(int(''.join(state_key), 3))[2:]

    def get_action(self, observation):
        #print("enter")
        state = observation.board #埋まっている部分の可能性は他のところに山分けする
        
        

        state_key = self.generate_state_key(observation)
        #おけない部分の修正
        #山分けは誤差でばグルのでやめる
        p = 0
        tmp = {0:-1, 1:-1, 2:-1, 3:-1, 4:-1, 5:-1, 6:-1}
        for i in range(self.action_size):
            #print("state", state[i])
            if state[i] != 0:
                #print("bpi", self.pi[1])
                tmp[i] = 0
                #print("pi", self.pi[1])
            else:
                
                p += 1
       # print("middle", p)
        
        for i in range(self.action_size):
            if state[i] == 0:
                tmp[i] = 1 / p
                
        self.pi[state_key] = tmp
        action_probs = self.pi[state_key]
        
        
        actions = list(action_probs.keys())
        probs = list(action_probs.values())
        
        
        #if p != self.action_size and sum(probs) != 1:
            #print("before")
            #print(probs, sum(probs), state, p, self.pi[1])
            #print("after")
        
        #print("b",sum(probs),probs)
        return np.random.choice(actions, p=probs).tolist()

    def add(self, observation, action, reward):
        state_key = self.generate_state_key(observation)
        data = (state_key, action, reward)
        self.memory.append(data)

    def reset(self):
        self.memory.clear()

    def eval(self):
        G = 0
        for data in reversed(self.memory):
            state_key, action, reward = data
            G = self.gamma * G + reward
            self.cnts[state_key] += 1
            self.v[state_key] += (G - self.v[state_key]) / self.cnts[state_key]


In [ ]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])
#　遷移確率は無い（完全にa,sに依存）し、
# 報酬もゲームが終了するか否かの判定によって得られるためQ関数を用いる必然性はないが…
class McAgent:
    def __init__(self):
        self.gamma = 1
        self.action_size = 7
        self.pi = 

In [134]:
env.reset()
# Play as the first agent against default "random" agent.
env.run([my_agent, "random"])
env.render(mode="ipython", width=500, height=450)

In [170]:
env = make("connectx", debug=True)
trainer = env.train([None, "random"])
episodes = 1
agent = RandomAgent()
print(type(agent))


for episode in range(episodes):
    #print("new game")
    agent.reset()
    observation = trainer.reset()
    #print(observation, agent.pi[1])
    env.render()
    
    
    while not env.done:
        #print("key", agent.generate_state_key(observation), agent.pi[agent.generate_state_key(observation)])
        action = agent.get_action(observation)
        #print(type(action.tolist()))
        next_observation, reward, done, info = trainer.step(action)
        #print(type(reward), reward,done,info)
        agent.add( observation, action, reward)
        
        if done:
            #print(reward)
            agent.eval()
            break
        
        observation = next_observation
        #print(observation.board)

#print(agent.pi)
#print(agent.memory)
#print(agent.v)

<class '__main__.RandomAgent'>


In [171]:
pi = agent.pi
p = {}

    
def my_agent(observation, configuration):
    import numpy as np
    from random import choice
    from collections import defaultdict
    pi = agent.pi # あとで変換
    state = observation.board
    state.append(observation.mark)
    state_key = np.array(state).astype(str) 
    state_key = hex(int(''.join(state_key), 3))[2:]
    if state_key not in pi.keys():
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    
    actions = list(pi[state_key].keys())
    probs = list(pi[state_key].values())
    action = np.random.choice(actions, p=probs).tolist()
    
    return action

print(dict(agent.pi))

{'1': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}, '12a': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}, '81d21': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}, '1be1b98': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}, '1bf1357': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.14285714285714285, 4: 0.14285714285714285, 5: 0.14285714285714285, 6: 0.14285714285714285}, '4797be24': {0: 0.14285714285714285, 1: 0.14285714285714285, 2: 0.14285714285714285, 3: 0.1428571428571428

In [173]:
def my_agent(observation, configuration):
    import numpy as np
    from random import choice
    pi = {'1':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'12a':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'81d21':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'1be1b98':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'1bf1357':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'4797be24':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'56fcc9fb':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'42191eead2':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'42191eedbd':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'2320792ad7a7a':{0:0.14285714285714285,1:0.14285714285714285,2:0.14285714285714285,3:0.14285714285714285,4:0.14285714285714285,5:0.14285714285714285,6:0.14285714285714285},'258093c2a9439fcf':{0:0.16666666666666666,1:0.16666666666666666,2:0.16666666666666666,3:0.16666666666666666,4:0,5:0.16666666666666666,6:0.16666666666666666},'258093ced6b13d4e':{0:0.16666666666666666,1:0.16666666666666666,2:0.16666666666666666,3:0.16666666666666666,4:0,5:0.16666666666666666,6:0.16666666666666666},'2580986e7cb68a03':{0:0.16666666666666666,1:0.16666666666666666,2:0.16666666666666666,3:0.16666666666666666,4:0,5:0.16666666666666666,6:0.16666666666666666},'2587aa94e15bc3aa':{0:0.16666666666666666,1:0.16666666666666666,2:0.16666666666666666,3:0.16666666666666666,4:0,5:0.16666666666666666,6:0.16666666666666666}}
    state = observation.board
    state.append(observation.mark)
    state_key = np.array(state).astype(str) 
    state_key = hex(int(''.join(state_key), 3))[2:]
    if state_key not in pi.keys():
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    
    actions = list(pi[state_key].keys())
    probs = list(pi[state_key].values())
    action = np.random.choice(actions, p=probs).tolist()
    
    return action
    

In [174]:
import sys
out = sys.stdout

sys.stdout = out

env = make("connectx", debug=True)
env.run([my_agent, my_agent])
print("Success" if env.state[0].status == env.state[1].status == "DONE" else "Failed")

Success


In [158]:
import sys
out = sys.stdout
sub = utils.read_file("/kaggle/working/submission.py")
print(callable("/kaggle/working/submission.py"))
agent = utils.call(submission)
print(agent)
print(type(agent))
sys.stdout = out

env = make("connectx", debug=True)
env.run([agent, agent])
print("Success" if env.state[0].status == env.state[1].status == "DONE" else "Failed")

False
None
<class 'NoneType'>
Invalid Action: Invalid column: 0
Failed


In [172]:
my_agent = '''def my_agent(observation, configuration):
    import numpy as np
    from random import choice
    pi = '''\
    + str(dict(agent.pi)).replace(' ', '')\
    + '''
    state = observation.board
    state.append(observation.mark)
    state_key = np.array(state).astype(str) 
    state_key = hex(int(''.join(state_key), 3))[2:]
    if state_key not in pi.keys():
        return choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    
    actions = list(pi[state_key].keys())
    probs = list(pi[state_key].values())
    action = np.random.choice(actions, p=probs).tolist()
    
    return action
    '''
with open('submission.py', 'w') as f:
    f.write(my_agent)

In [ ]:
env.reset()
# Play as the first agent against default "random" agent.
env.run([my_agent, "random"])
env.render(mode="ipython", width=500, height=450)

In [ ]:
import inspect
import os

def write_agent_to_file(function, file):
    with open(file, "a" if os.path.exists(file) else "w") as f:
        f.write(inspect.getsource(function))
        print(function, "written to", file)

write_agent_to_file(my_agent, "submission.py")